In [3]:
# Preliminaries

# scratch_location = r'/scratch/hmnshpl'
import os
import sys
import heapq
import getpass
import numpy as np
import pandas as pd
import networkx as nx
from copy import deepcopy
from tqdm import tqdm
from collections import defaultdict, Counter

dataset_name = 'wikipedia'
scratch_location = rf'/scratch/{getpass.getuser()}'


## Load Data
# Load data and train val test split
graph_df = pd.read_csv('{}/processed_data/{}/ml_{}.csv'.format(scratch_location,
                                                            dataset_name,
                                                            dataset_name)
                    )
edge_raw_features = np.load('{}/processed_data/{}/ml_{}.npy'.format(scratch_location,
                                                                    dataset_name,
                                                                    dataset_name)
                            )
node_raw_features = np.load('{}/processed_data/{}/ml_{}_node.npy'.format(scratch_location,
                                                                        dataset_name,
                                                                        dataset_name)
                            )

# Set the working directory to the project root
project_root = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')) # this might cause issue
sys.path.append(project_root)

### Temperal EdgeRank Implementation

In [2]:
def temporal_pagerank_heap_np(E, beta, alpha, check_evolution=False):
    # print('\t inside tpr heap method')
    # Convert edges to a NumPy array
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    
    # Get the maximum node index to size the r and s arrays appropriately
    max_node = max(E['u'].max(), E['v'].max())
    
    # Initialize r and s arrays
    r = np.zeros(max_node + 1)
    s = np.zeros(max_node + 1)
    
    ts_tpr = [] if check_evolution else None
    
    # Use a heap to efficiently process edges in time order
    heap = [(t, u, v) for u, v, t in E]
    heapq.heapify(heap)
    # print('\t heapify successful')
    while heap:
        t, u, v = heapq.heappop(heap)
        
        # Update r and s values
        delta = 1 - alpha
        r[u] += delta
        s[u] += delta
        r[v] += s[u] * alpha
        
        if beta < 1:
            s_v_increment = s[u] * (1 - beta) * alpha
            s[v] += s_v_increment
            s[u] *= beta
        else:
            s[v] += s[u] * alpha
            s[u] = 0
        
        # Store evolution if required
        if check_evolution:
            # ts_tpr.append((t, r.copy()))  # Store r values at current timestamp
            # Normalize r before appending
            total_r = r.sum()
            if total_r > 0:
                ts_tpr.append((t, r.copy() / total_r))
    # print('\t out of loop.')
    
    # Normalize r
    total_r = r.sum()
    if total_r > 0:
        r /= total_r
    
    if check_evolution:
        ts_tpr = np.array(ts_tpr, dtype=[('t', float), ('r', float, max_node + 1)])
    
    return r, ts_tpr

def compute_temporal_outgoing_degree(E):
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    outgoing_degree = defaultdict(int)
    temporal_outgoing_degree = defaultdict(list)
    heap = [(t, u, v) for u, v, t in E]
    heapq.heapify(heap)
    while heap:
        t, u, v = heapq.heappop(heap)
        outgoing_degree[u] += 1
        for node in outgoing_degree:
            temporal_outgoing_degree[node].append((t, outgoing_degree[node]))
    return temporal_outgoing_degree

### Ignore

In [ ]:
# Extract nodes, edges, and timestamps
edges = graph_df[['u', 'i', 'ts']].values
nodes = np.unique(edges[:, :2])  # Get unique nodes from edges

# Convert E to a more readable format if needed
edges_new = [(int(u), int(v), float(t)) for u, v, t in edges]

beta = 0.85
alpha = 0.15

r2, ts_tpr= temporal_pagerank_heap_np(edges_new, beta, alpha, True)
print(r2)

In [ ]:
def compute_temporal_outgoing_degree(E):
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    outgoing_degree = defaultdict(int)
    temporal_outgoing_degree = defaultdict(list)
    heap = [(t, u, v) for u, v, t in E]
    heapq.heapify(heap)
    while heap:
        t, u, v = heapq.heappop(heap)
        outgoing_degree[u] += 1
        for node in outgoing_degree:
            temporal_outgoing_degree[node].append((t, outgoing_degree[node]))
    return temporal_outgoing_degree

In [ ]:
from tqdm import tqdm


def compute_temporal_edgerank(E, beta, alpha):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    print(len(ts_tpr))
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    for tpr in tqdm(ts_tpr, desc='Calculating Temporal EdgeRank'):
        t, r = tpr['t'], tpr['r']
        for u, v, t_ev in E:
            if t_ev <= t:
                if u in r and u in temporal_outgoing_degree:
                    current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
                    if current_degree > 0:
                        edge_rank = r[u] / current_degree
                    else:
                        edge_rank = 0
                    print('Add rank...')
                    temporal_edgerank[u][v].append((t, edge_rank))
    
    return temporal_edgerank

In [ ]:
# Example usage:
E = [
    (1, 2, 1.0),
    (2, 3, 2.0),
    (1, 3, 3.0),
    (1, 2, 4.0),
    (2, 1, 5.0)
]

temporal_edgerank = compute_temporal_edgerank(E, beta, alpha)

# Print Temporal EdgeRank
for u in temporal_edgerank:
    for v in temporal_edgerank[u]:
        print(f"Edge ({u}, {v}) Temporal EdgeRank over time: {temporal_edgerank[u][v]}")

In [ ]:
_, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
ts_tpr

In [ ]:
temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
temporal_outgoing_degree

In [ ]:
temporal_edgerank = defaultdict(lambda: defaultdict(list))

for tpr in tqdm(ts_tpr, desc='Calculating Temporal EdgeRank'):
    t, r = tpr['t'], tpr['r']
    for u, v, t_ev in E:
        if t_ev <= t:
            if u in temporal_outgoing_degree:
                current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
                if current_degree > 0:
                    edge_rank = r[u] / current_degree
                else:
                    edge_rank = 0
                print('Add rank...')
                temporal_edgerank[u][v].append((t, edge_rank))
                
len(temporal_edgerank)

In [ ]:
for u, v, t_ev in E:
    r = dict(ts_tpr)[t_ev]
    if u in temporal_outgoing_degree:
        current_degree = [d for time, d in temporal_outgoing_degree[u] if time <= t][-1]
        if current_degree > 0:
            edge_rank = r[u] / current_degree
        else:
            edge_rank = 0
        print('Add rank...')
        temporal_edgerank[u][v].append((t, edge_rank))
len(temporal_edgerank)        

In [ ]:
temporal_edgerank

In [ ]:
temporal_edgerank

In [ ]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'for {t=}')
        for u, v, t_ev in E:
            print(f'\tProcessing {t_ev=} for {(u, v)}', end=' ')
            if t_ev <= t:
                print('started: ')
                current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]
                if current_degrees:
                    current_degree = smooth_degree(current_degrees[-1])
                    if current_degree > 0:
                        time_diff = t - t_ev
                        edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                    else:
                        edge_rank = 0
                    temporal_edgerank[u][v].append((t, edge_rank))
                    print(f'\t\tappended {(u, v)} for time {t}')
            else:
                print('Unprocessed.', end='\n')
    assert len(temporal_edgerank) != 0, 'Check2'
    
    return temporal_edgerank

In [ ]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))

    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'Processing timestamp {t}')
        
        processed_edges = set()  # Track processed (u, v, t_ev) tuples
        
        for u, v, t_ev in E:
            print(f'\t{(u, v, t_ev)}', end = ' ')
            if (u, v, t_ev) in processed_edges:
                print(f' already processed. Skipped...')
                continue  # Skip if already processed
            # print('')
            if t_ev <= t: # changed here from t_ev <= t to t_ev == t
                print(f'\tProcessing edge ({u}, {v}) at time {t_ev} for timestamp {t}')
                
                # Retrieve degrees up to current timestamp t
                current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]  # changed here from time <= t to time == t
                if current_degrees:
                    current_degree = smooth_degree(current_degrees[-1])
                    if current_degree > 0:
                        time_diff = t - t_ev
                        edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                    else:
                        edge_rank = 0
                    temporal_edgerank[u][v].append((t, edge_rank))
                    print(f'\t\t\t\t\tAppended edge rank: ({u}, {v}, {t}, {edge_rank})')
                else:
                    print(f'\t\tNo valid degree for user {u} at time {t}')
                
                processed_edges.add((u, v, t_ev))  # Mark this edge as processed
            else:
                print(f'\tSkipping edge ({u}, {v}) at time {t_ev} as it is not != {t}')
    
    assert len(temporal_edgerank) != 0, 'Check2'
    return temporal_edgerank

In [ ]:
def compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9):
    _, ts_tpr = temporal_pagerank_heap_np(E, beta, alpha, check_evolution=True)
    temporal_outgoing_degree = compute_temporal_outgoing_degree(E)
    assert len(ts_tpr) > 0, 'Check1'
    assert len(temporal_outgoing_degree) > 0, 'Check2'
    temporal_edgerank = defaultdict(lambda: defaultdict(list))
    
    def smooth_degree(degree):
        return degree ** 0.5  # Smoothing function (e.g., square root)
    
    # Create a mapping from edges to their timestamps
    edge_to_time = defaultdict(list)
    for u, v, t_ev in E:
        edge_to_time[(u, v)].append(t_ev)
    
    for tpr in ts_tpr:
        t, r = tpr['t'], tpr['r']
        print(f'Processing timestamp {t}')
        
        for (u, v), times in edge_to_time.items():
            # Process each edge only if it has not been processed for this timestamp
            for t_ev in times:
                if t_ev <= t:
                    print(f'\tProcessing edge ({u}, {v}) at time {t_ev} for timestamp {t}')
                    
                    # Retrieve degrees up to current timestamp t
                    current_degrees = [d for time, d in temporal_outgoing_degree[u] if time <= t]
                    if current_degrees:
                        current_degree = smooth_degree(current_degrees[-1])
                        if current_degree > 0:
                            time_diff = t - t_ev
                            edge_rank = (r[u] / current_degree) * (decay_factor ** time_diff)
                        else:
                            edge_rank = 0
                        temporal_edgerank[u][v].append((t, edge_rank))
                        print(f'\t\t\t\t\tAppended edge rank: ({u}, {v}, {t}, {edge_rank})')
                    
                    # Stop processing this edge after the first valid timestamp
                    break
                else:
                    print(f'\tSkipping edge ({u}, {v}) at time {t_ev} as it is not <= {t}')
    
    assert len(temporal_edgerank) != 0, 'Check2'
    return temporal_edgerank


In [ ]:
temporal_edgerank = compute_temporal_edgerank_with_time_decay_smoothening(E, beta, alpha, decay_factor=0.9)
# (temporal_edgerank)

In [ ]:
E

In [ ]:
# Function to sort the inner defaultdict
def sort_inner_dict(d):
    return {k: sorted(v, key=lambda x: x[0]) for k, v in d.items()}

# Sort the inner default dicts
sorted_data = {k: sort_inner_dict(v) for k, v in temporal_edgerank.items()}

# Sort the outer defaultdict
sorted_data = dict(sorted(sorted_data.items()))

print(sorted_data)


In [ ]:
# Flatten the defaultdict into a list of records [u, i, ts, value]
records = []
for u, inner_dict in temporal_edgerank.items():
    for i, values in inner_dict.items():
        for ts, value in values:
            records.append([u, i, ts, value])

# Create a DataFrame from the records
df = pd.DataFrame(records, columns=['u', 'i', 'ts', 'value'])

print(df)

In [ ]:
df2 = pd.DataFrame(E)
# Rename the columns in df2 to match those in df1
df2.columns = ['u', 'i', 'ts']

# Merge the DataFrames on 'u', 'i', and 'ts'
merged_df = pd.merge(df2, df, on=['u', 'i', 'ts'])

(merged_df)

In [ ]:
len(df2), len(merged_df)

In [ ]:
df2

### New tests

In [ ]:
# Extract nodes, edges, and timestamps
edges = graph_df[['u', 'i', 'ts']].values
nodes = np.unique(edges[:, :2])  # Get unique nodes from edges

# Convert E to a more readable format if needed
edges_new = [(int(u), int(v), float(t)) for u, v, t in edges]

beta = 0.85
alpha = 0.15

In [ ]:
r2, ts_tpr= temporal_pagerank_heap_np(edges_new, beta, alpha, True)
# print(r2)

In [ ]:
# temporal_outgoing_degree = compute_temporal_outgoing_degree(edges_new)
# temporal_outgoing_degree

In [ ]:
# temporal_outgoing_degree[1]

In [ ]:
# def compute_temporal_outgoing_degree(E):
#     E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
#     outgoing_degree = defaultdict(int)
#     temporal_outgoing_degree = defaultdict(list)
    
#     # Sort the edges by time
#     E = np.sort(E, order='t')
    
#     for edge in E:
#         u, t = edge['u'], edge['t']
#         outgoing_degree[u] += 1
#         for node, degree in outgoing_degree.items():
#             temporal_outgoing_degree[node].append((t, degree))
    
#     return temporal_outgoing_degree

In [ ]:
# temporal_outgoing_degree = compute_temporal_outgoing_degree(edges_new)
# temporal_outgoing_degree

In [ ]:
from collections import Counter
import numpy as np

def compute_overall_outgoing_degree(E):
    # Convert E to a numpy array if it's not already
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    
    # Use Counter to count the occurrences of each source node
    outgoing_degree = Counter(E['u'])
    
    return dict(outgoing_degree)

In [ ]:
temporal_outgoing_degree = compute_overall_outgoing_degree(edges_new)
len(temporal_outgoing_degree)

In [ ]:
print(len(ts_tpr))
print(len(ts_tpr[1]))

In [ ]:
type(ts_tpr)

In [ ]:
# Extract timestamps and PageRank values
timestamps, pagerank_arrays = zip(*ts_tpr)
timestamps = np.array(timestamps)
pagerank_arrays = np.array(pagerank_arrays)

In [ ]:
pagerank_arrays.shape, len(graph_df['ts']),  len(graph_df['u'].unique())

In [ ]:
type(temporal_outgoing_degree)

for key, val in temporal_outgoing_degree.items():
    print(key, val)
    break

In [ ]:
# ts_to_node_dict = dict(zip(*graph_df[['u', 'ts']]))
# ts_to_node_dict

ts_to_node_dict = graph_df[['u', 'ts']].set_index('ts').to_dict()['u']

In [ ]:
ter_dict = {}
# ts_to_node_dict = graph_df[['u', 'ts']].set_index('ts').to_dict()['u']
ts_to_node_dict = graph_df.groupby('ts')['u'].apply(list).to_dict()

cntr = 0
for i in tqdm(range(len(timestamps)), desc='Calculating TER'):
    if cntr == 10:
        break
    r = pagerank_arrays[i]
    ts = timestamps[i]
    node_list = ts_to_node_dict[ts]
    # tpr_t = [r[node] for node in node_list]
    # outgoing_degree_node = [temporal_outgoing_degree[node] for node in node_list]
    
    # ter_dict[ts] = tpr_t / outgoing_degree_node
    ter = [r[node] / temporal_outgoing_degree[node] for node in node_list]
    
    ter_dict[ts] = np.sum(ter)  # can we use mean here?
    # cntr+=1
    

In [ ]:
def is_strictly_increasing(alist):
    return all(x < y for x, y in zip(alist, alist[1:]))



is_strictly_increasing(list(ter_dict.values()))

# sorted(ter_dict.items(), key = lambda x: x[1])

sorted_dict = dict(sorted(ter_dict.items(), key=lambda x: x[1], reverse=True))
sorted_dict

In [ ]:
# len(ter_dict) how do I account for multiple nodes

### Final Testing

In [11]:
from collections import Counter
import numpy as np

def compute_overall_outgoing_degree(E):
    # Convert E to a numpy array if it's not already
    E = np.array(E, dtype=[('u', int), ('v', int), ('t', float)])
    
    # Use Counter to count the occurrences of each source node
    outgoing_degree = Counter(E['u'])
    
    return dict(outgoing_degree)


def calculate_temporal_edge_rank(_graph_df, beta = 0.85, alpha = 0.15):
    graph_df = _graph_df.copy(deep=True)
    
    # Extract nodes, edges, and timestamps
    edges = graph_df[['u', 'i', 'ts']].values
    
    # Convert E to a more readable format if needed
    edges_new = [(int(u), int(v), float(t)) for u, v, t in edges]
    
    _, ts_tpr= temporal_pagerank_heap_np(edges_new, beta, alpha, True)
    temporal_outgoing_degree = compute_overall_outgoing_degree(edges_new)
    
    # Extract timestamps and PageRank values
    timestamps, pagerank_arrays = zip(*ts_tpr)
    timestamps = np.array(timestamps)
    pagerank_arrays = np.array(pagerank_arrays)
    
    ts_to_node_dict = graph_df.groupby('ts')['u'].apply(list).to_dict()
    
    ter_dict = {}
    
    for i in tqdm(range(len(timestamps)), desc='Calculating TER'):
        r = pagerank_arrays[i]
        ts = timestamps[i]
        node_list = ts_to_node_dict[ts]
        ter = [r[node] / temporal_outgoing_degree[node] for node in node_list]
        
        ter_dict[ts] = np.sum(ter)  # can we use mean here?
    
    return ter_dict

In [5]:
new_ter_dict = calculate_temporal_edge_rank(graph_df)
len(new_ter_dict)

Calculating TER: 100%|██████████| 157474/157474 [00:01<00:00, 148509.61it/s]


152757

In [18]:
from collections import Counter
import numpy as np
from tqdm import tqdm
import heapq

def compute_overall_outgoing_degree(E):
    return Counter(E['u'])

def calculate_temporal_edge_rank(_graph_df, beta=0.85, alpha=0.15):
    graph_df = _graph_df.copy(deep=True)
    
    # Extract nodes, edges, and timestamps as a list of tuples
    edges = graph_df[['u', 'i', 'ts']].values.tolist()
    edges = [(int(u), int(v), float(t)) for u, v, t in edges]
    
    _, ts_tpr = temporal_pagerank_heap_np(edges, beta, alpha, True)
    
    # Create numpy array for efficient operations
    edges_array = np.array(edges, dtype=[('u', int), ('v', int), ('t', float)])
    temporal_outgoing_degree = compute_overall_outgoing_degree(edges_array)
    
    # Extract timestamps and PageRank values
    timestamps, pagerank_arrays = zip(*ts_tpr)
    timestamps = np.array(timestamps)
    pagerank_arrays = np.array(pagerank_arrays)
    
    ts_to_node_dict = graph_df.groupby('ts')['u'].apply(np.array).to_dict()
    
    ter_dict = {}
    
    for i in tqdm(range(len(timestamps)), desc='Calculating TER'):
        r = pagerank_arrays[i]
        ts = timestamps[i]
        node_list = ts_to_node_dict[ts]
        ter = r[node_list] / np.vectorize(temporal_outgoing_degree.__getitem__)(node_list)
        ter_dict[ts] = np.sum(ter)
    
    return ter_dict

In [19]:
new_ter_dict = calculate_temporal_edge_rank(graph_df)
len(new_ter_dict)

Calculating TER: 100%|██████████| 157474/157474 [00:04<00:00, 33392.51it/s]


152757

### New attempts for a comprehensive TER

In [31]:
# Function to compute the global Temporal PageRank
def compute_global_temporal_pagerank(edges, beta=0.85, alpha=0.15):
    # Use your existing Temporal PageRank calculation method (e.g., temporal_pagerank_heap_np)
    # This should return a dictionary with nodes as keys and global TPR as values
    global_tpr, _ = temporal_pagerank_heap_np(edges, beta, alpha, False)
    return global_tpr

# Function to compute timestamp-specific Temporal PageRank
def compute_timestamp_specific_temporal_pagerank(edges, beta=0.85, alpha=0.15):
    # Use your existing Temporal PageRank calculation method with timestamp-specific mode
    # This should return a list of (timestamp, dict) where dict contains node-specific TPR
    _, ts_tpr = temporal_pagerank_heap_np(edges, beta, alpha, True)
    return ts_tpr

# Function to compute the outgoing degree of nodes globally
def compute_global_outgoing_degree(edges):
    edges = np.array(edges, dtype=[('u', int), ('v', int), ('t', float)])
    u_values = edges['u'].tolist()
    return dict(Counter(u_values))

# Function to compute the outgoing degree of nodes at specific timestamps
def compute_timestamp_specific_outgoing_degree(edges):
    edges_by_ts = defaultdict(list)
    for u, v, t in edges:
        edges_by_ts[t].append((u, v))
    
    ts_outgoing_degree = {}
    for t, edge_list in tqdm(edges_by_ts.items()):
        ts_outgoing_degree[t] = dict(Counter([u for u, _ in edge_list]))
    
    return ts_outgoing_degree


def compute_cumulative_timestamp_specific_outgoing_degree(graph_df):
    # Sort by timestamp to ensure cumulative counting is correct
    graph_df = graph_df.sort_values('ts')
    
    # Initialize cumulative outgoing degree dictionary
    cumulative_outgoing_degree = defaultdict(int)
    
    # Initialize result dictionary
    ts_outgoing_degree = {}
    
    # Iterate through the DataFrame row by row
    for ts, group in tqdm(graph_df.groupby('ts'), desc="Calculating cumulative outgoing degree"):
        # Update cumulative counts
        for u in group['u']:
            cumulative_outgoing_degree[u] += 1
            
        # Store the current cumulative counts in the result dictionary
        ts_outgoing_degree[ts] = dict(cumulative_outgoing_degree)
    
    return ts_outgoing_degree



# Function to compute Temporal EdgeRank for each edge
def compute_temporal_edgerank(graph_df, beta=0.85, alpha=0.15, gamma=0.5):
    edges_list = graph_df[['u', 'i', 'ts']].values
    edges = np.array(
        list(zip(edges_list[:, 0].astype(int), edges_list[:, 1].astype(int), edges_list[:, 2].astype(float))),
        dtype=[('u', int), ('v', int), ('t', float)]
    )
    
    # edges
    print(f'Calculating TPR...', end='\r')
    global_tpr, ts_tpr = temporal_pagerank_heap_np(edges, beta, alpha, True)
    # Convert ts_tpr to a dictionary with timestamps as keys
    ts_tpr = {row['t']: row['r'] for row in ts_tpr}
    print(f'Calculated TPR      ')
    
    # ts_tpr = compute_timestamp_specific_temporal_pagerank(edges, beta, alpha)  # redundant
    print(f'Calculating global_out_deg...', end='\r')
    global_out_deg = compute_global_outgoing_degree(edges)
    print(f'Calculated global_out_deg      ')
    print(f'Calculating cumulative_timestamp_specific_outgoing_degree...', end='\r')
    ts_out_deg = compute_cumulative_timestamp_specific_outgoing_degree(graph_df[['u', 'i', 'ts']])
    print(f'Calculated cumulative_timestamp_specific_outgoing_degree       ')
    
    ter_dict = {}

    for u, _, t in tqdm((edges), desc='Processing'):
        TER_global = global_tpr[u] / global_out_deg[u]
        TER_ts = ts_tpr[t][u] / ts_out_deg[t][u]
        ter_ts = gamma * TER_global + (1 - gamma) * TER_ts
        ter_dict[t] = ter_ts
        
    return ter_dict

# Main function to call and calculate Combined Temporal EdgeRank
def calculate_combined_temporal_edgerank(graph_df, beta=0.85, alpha=0.15, gamma=0.5):
    
    return compute_temporal_edgerank(graph_df, beta, alpha, gamma)


In [21]:
combined_ter_dict = calculate_combined_temporal_edgerank(graph_df=graph_df)

Calculated TPR      
Calculated global_out_deg      


Calculating cumulative outgoing degree: 100%|██████████| 152757/152757 [00:53<00:00, 2864.52it/s]


Calculated cumulative_timestamp_specific_outgoing_degree       


Processing: 100%|██████████| 157474/157474 [00:01<00:00, 123214.32it/s]


In [25]:
len(combined_ter_dict), len(graph_df['ts'])

(152757, 157474)

157474